# Wavetable

A few experiments with the Wavetable crate.

First we need to do some setup to use Plotly for displaying graphs.

In [44]:
:dep plotly = { version = ">=0.6.0" }
:dep itertools-num = "0.1.3"

In [45]:
extern crate plotly;
extern crate rand_distr;
extern crate itertools_num;
extern crate itertools;

In [49]:
// TODO: We probably don't need all of these, clean up
use itertools_num::linspace;
use plotly::{Bar, NamedColor, Plot, Rgb, Rgba, Scatter};
use plotly::common::{ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode, Side, Title};
use plotly::layout::{Axis, BarMode, GridPattern, Layout, LayoutGrid, Legend, RowOrder, TicksDirection};
use rand_distr::{Distribution, Normal, Uniform};

Then we load the Wavetable crate and create some handler objects.

In [5]:
:dep wavetable = { path = "/Users/ingo/Documents/Programming/src/rust/wavetable/" }

In [47]:
use wavetable::{WtReader, WtManager};

let mut wt_manager = WtManager::new(44100.0);
let basic_wave_id = 0;
wt_manager.add_basic_tables(basic_wave_id);

## Examining the basic wave shapes

Create a plot of the basic wave shapes (sine, triangle, ramp, square)

In [70]:
let wt_basic = wt_manager.get_table(basic_wave_id).unwrap();

// Prepare plot data
let num_tables = wt_basic.num_tables;
let num_samples = wt_basic.num_samples;
let n = num_samples * num_tables;
let t: Vec<f64> = linspace(0., n as f64, n).collect();
let mut y: Vec<f32> = vec!();
for i in 0..num_tables {
    // TODO: There's got to be a better way
    let mut temp = wt_basic.get_wave(i).clone();
    for j in 0..num_samples {
        y.push(temp[j]);
    }
}

// Create the plot
let trace = Scatter::new(t, y);
let mut plot = Plot::new();
plot.add_trace(trace);
plot.set_layout(Layout::new());
plot.notebook_display();

Take the tables and run an FFT on them, show the first 32 of 1024 harmonics.

In [66]:
let harmonics = wt_basic.convert_to_harmonics();
let n = 32;

let t: Vec<f64> = linspace(0., n as f64, n).collect();
let trace_1 = Scatter::new(t.clone(), harmonics[0].clone());
let trace_2 = Scatter::new(t.clone(), harmonics[1].clone()).x_axis("x2").y_axis("y2");
let trace_3 = Scatter::new(t.clone(), harmonics[2].clone()).x_axis("x3").y_axis("y3");
let trace_4 = Scatter::new(t, harmonics[3].clone()).x_axis("x4").y_axis("y4");
    
let mut plot = Plot::new();
plot.add_trace(trace_1);
plot.add_trace(trace_2);
plot.add_trace(trace_3);
plot.add_trace(trace_4);

let layout = Layout::new()
    .grid(LayoutGrid::new().rows(2).columns(2).pattern(GridPattern::Independent),)
    .height(800);
plot.set_layout(layout);
plot.notebook_display();

Creating 4 lists of harmonics with 1024 entries each for waves with 2048 samples
